# Rasterio: Reading and Writing Data, Descriptive Information  

When you're working in with rasters in Rasterio, a typical workflow is often reading data, doing something to it, and then writing out a new dataset. To perform these i/o type tasks, accessing descriptive info about the dataset is also essential. 

Start by importing libraries:  
```python
import numpy as np
import rasterio as rio
from rasterio.plot import show
import os
```

Important: only run this once or you'll wind up in the wrong directory.
```python
os.chdir('../')
os.getcwd()
```

First, we'll read in a raster as a Rasterio dataset object:  

```python
src = rio.open('workshopdata/Flatirons_DEM_1m.tif')
```

Just so you know what you're looking at... 

```python
show(src)
```

More visualization later...  

#### Let's go over some descriptive attributes:  

```python
src.name
```

```python
src.width
```

```python
src.height
```

Check the coordinate reference system (CRS)
```python
src.crs
```

How many bands are in this image?  
```python
src.count
```

The unit is part of the coordinate system:
```python
src.crs.linear_units
```

Now, let's get the extent AKA bounds:
```python
src.bounds
```

Affine transformation:
```python
src.transform
```

Transform is super important, but what the heck is it?  

(cell width, x origin, x origin coordinate, y origin, cell height, y origin coordinate)

**From Rasterio's docs:** 
A dataset’s `DatasetReader.transform` is an [affine transformation matrix](https://en.wikipedia.org/wiki/Affine_transformation) that maps pixel locations in (row, col) coordinates to (x, y) spatial positions. The product of this matrix and (0, 0), the row and column coordinates of the upper left corner of the dataset, is the spatial position of the upper left corner.  
  
Upper left corner position is (0,0), so...  
```python
src.transform * (0,0)
```

There's the coordinates of our top left corner...  

For funsies, we could take a stab at the center point, or at least pretty darn close:
```python
centerX = src.width/2
centerY = src.height/2
```

```python
src.transform * (centerX,centerY)
```

So, that's how you apply a transformation--it does the math over the pixels to "georeference" them to a coordinate system, or change it from one crs to another... which we'll work on in the next notebook.  

You can access parts of the transform as well, such as cell dimensions:
```python
cellWidth = src.transform[0]
cellHeight = src.transform[4]
```

```python
cellWidth, cellHeight
```

Okay, and finally two related and important descriptive tools:
```python
src.meta
```

And similar...  

```python
src.profile
```

Note that you can access these:  
```python
src.meta['dtype']
```

Now let's do a little bit of work on the raster layer and save it's output to a new file.  

Frequently, Numpy is used to work with arrays--and a raster layer is just a big ol' array...  

To read a Rasterio dataset object as a Numpy array use src.read(). The parameter is the band you want to read:

```python
array = src.read(1)
```

And here it is... 

```python
array
```

Now, let's do something super basic for now: multiply every value in the array times 2...   
```python
array2 = array * 2
```

Did it work?

```python
print(array.max())
print(array2.max())
```

Complex analysis, right?  

Let's grab the height and width of our array2 and save them for later... note that these should be exactly the same as our source DEM, so in this case, you could also use src.width and src.height...  

But we'll use the Numpy method:
```python
height = np.size(array2,0)
width = np.size(array2,1)
```

Okay, let's take our new data and write it to a new GeoTiff file.  

The steps are: 
1. Open a new array in write mode--with all the correct parameters... 
2. Write the array into the Rasterio dataset object
3. Close it

So, begin by creating and opening a new GeoTiff that will store our data...  

First let's take a look at the documentation: https://rasterio.readthedocs.io/en/latest/quickstart.html#opening-a-dataset-in-writing-mode

The parameters we need are:
- driver: the name of the desired format driver
- width: the number of columns of the dataset
- height: the number of rows of the dataset
- count: a count of the dataset bands
- dtype: the data type of the dataset
- crs: a coordinate reference system identifier or description
- transform: an affine transformation matrix  

Note that several of these are optional, but you probably want to use at least these. And there are more optional params as well...  

For starters let's do this:

```python
new = rio.open('new_dem.tif', 'w', 
               driver='GTiff', 
               height = height, 
               width = width, 
               count = 1, 
               dtype = src.meta['dtype'], 
               crs = src.crs, 
               transform = src.transform)
```

Now write the array to the open data set:
```python
new.write(array2,1)
```

Close it when you're done:  
```
new.close()
```

#### That's it!   
The basic gist is that you need the descriptive info (meta, profile, crs, transform, width, height, dtype, etc) to write a new dataset.  

#### However!  
There is a slightly more streamlined approach where you use the meta/profile of the source image and repurpose it (and modify if needed) and use it in the destination image....  

First, note that the `.meta` and `.profile` information has all the same parameters that you need to open a brand new dataset in write mode:  

```python
src.meta
```

Same stuff!  

Okay, make a copy of it to a new variable, we'll call it kwargs, which is short for key word arguments:
```python
kwargs = src.meta.copy()
```

Okay, now we have all the parameters necessary saved in a dict, which is of course, key:value pairs....  

In this case, none of them need updating because all we did was double the values in the array, we didn't modify it's shape at all so... 

Just open a new Rasterio data oject in write mode and apply the kwargs like this:
```python
new = rio.open('new_dem.tif', 'w', **kwargs)
```

That's called a **double star expression**--it is used to unpack keyword arguments as parameters of a function.  

And now, same as before, write and close:
```
new.write(array2,1)
new.close()
```

### DONE!